In [49]:
import pandas as pd
import requests
import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [50]:
city="Delhi"
api_key = os.getenv("WEATHER_API_KEY")
url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
response = requests.get(url)

In [51]:
data = response.json()
print(data)

{'coord': {'lon': 77.2167, 'lat': 28.6667}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 33.05, 'feels_like': 35.29, 'temp_min': 33.05, 'temp_max': 33.05, 'pressure': 1002, 'humidity': 46, 'sea_level': 1002, 'grnd_level': 977}, 'visibility': 7000, 'wind': {'speed': 3.09, 'deg': 300}, 'clouds': {'all': 2}, 'dt': 1758802674, 'sys': {'type': 1, 'id': 9165, 'country': 'IN', 'sunrise': 1758760854, 'sunset': 1758804289}, 'timezone': 19800, 'id': 1273294, 'name': 'Delhi', 'cod': 200}


In [52]:
with open("weather_data.json",'w') as file:
    json.dump(data,file,indent=4)

In [53]:
weather_params = ['temp','pressure','humidity','sea_level']

In [54]:
for i in weather_params:
    print(f"{i}: {data['main'][f'{i}']}")

temp: 33.05
pressure: 1002
humidity: 46
sea_level: 1002


#### Inserting data into postgressql

In [ ]:
import psycopg2
from psycopg2 import sql
import dotenv
from dotenv import load_dotenv
load_dotenv()

#connect to posgresql
conn = psycopg2.connect(
    host=f"{os.getenv("HOST")}",
    database="weather_reatail_db",
    user=f"{os.getenv("POSTGRES_USER")}",
    password=f"{os.getenv("POSTGRES_PASSWORD")}"
)

cursor = conn.cursor()

In [63]:
cursor.execute(
    """CREATE TABLE new_weather(
        id SERIAL PRIMARY KEY,
        country CHAR(2),
        city VARCHAR(20),
        weather_main VARCHAR(50),
        weather_description VARCHAR(100),
        temp REAL,
        temp_min REAL,
        temp_max REAL,
        pressure REAL,
        humidity REAL,
        visibility INT,
        wind_speed REAL,
        clouds INT,
        inserted_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
)

In [62]:
conn.rollback()   # reset the transaction

In [64]:
conn.commit()

In [65]:
weather_tuple = (
    data['sys']['country'],
    data['name'],
    data['weather'][0]['main'],
    data['weather'][0]['description'],
    data['main']['temp'],
    data['main']['temp_min'],
    data['main']['temp_max'],
    data['main']['pressure'],
    data['main']['humidity'],
    data['visibility'],
    data['wind']['speed'],
    data['clouds']['all'],
)

In [66]:
cursor.execute(
    """
    INSERT INTO new_weather
    (country, city, weather_main, weather_description, temp, temp_min, temp_max, pressure, humidity, visibility, wind_speed, clouds)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """,weather_tuple
)

In [67]:
conn.commit()